In [43]:
import pandas as pd
import tqdm
import re
import kss

from collections import Counter

from konlpy.corpus import kolaw
from konlpy.tag import Okt
from konlpy.utils import concordance, pprint
from matplotlib import pyplot

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
# nltk.download()

import warnings
warnings.filterwarnings('ignore')

In [2]:
book = pd.read_csv('뉴인터파크1000권.csv', index_col=0)
story = pd.read_csv('소설줄거리.csv')

In [49]:
# df = pd.concat([book, story], axis=1)
df = pd.read_csv('novel_story.csv')


In [54]:
df[3:4]['story'] = '2022 부커상 최종 후보 지명작! 한국 호러 SF/판타지 대표작가 정보라 대표작 2022년 인터내셔널 부커상 1차 후보가 발표되었을 때 한국 문학계는 몹시 놀랐다. 첫 번째 이유는 사상 최초로 한국 소설이 두 편이나 노미네이트되었기 때문이었으며, 두 번째 이유는 그 두 편 중 하나가 다른 하나에 비해 국내 문학 독자들에게 많이 알려지지 않은 소설집이었기 때문이었다. 어느 기자는 ‘무명의 부커상 후보’라는 단어를 써서 작가를 소개하기도 했다(SF계에서는 ‘어째서 정보라가 무명이냐’라며 탄식을 뱉긴 했으나). 그리고 최종 후보가 발표되었다. 그 ‘무명 아닌 무명’ 작가 정보라의 소설집, 《저주토끼》가 이름을 올렸다. 부커상 심사위원회는 《저주토끼》에 대해 “마법적 사실주의, 호러, SF의 경계를 초월했다”, “현대 사회에서 가부장제와 자본주의의 매우 현실적인 공포와 잔인함을 다루기 위해 환상적이고 초현실적인 요소들을 사용한다”라고 평했다. 관습과 허식에 얽매이지 않고 오래도록 자기만의 영역을 구축해 온 정보라의 쓸쓸한 이야기, 잔혹한 유머, ‘정보라’라고 이름 붙일 수밖에 없는 장르의 정수가 《저주토끼》에 있다. 할아버지는 늘 말씀하셨다. “저주에 쓰이는 물건일수록 예쁘게 만들어야 하는 법이다.” 대를 이어 저주 용품을 만드는 집안에서 태어난 손자와 그 할아버지의 이야기. 할아버지는 오래 전 젊은 나이에 생을 마감한 친구가 있었다. 할아버지의 친구네 집안은 마을 유지인 술도가. 바른 마음으로 좋은 전통주를 제조해서 팔려고 애쓰는 할아버지 친구네 집안사람들은 그러나 세상 돌아가는 약삭빠른 이치를 깨닫지 못했다. 정부 인사와의 친분, 인물, 접대, 뇌물은 뒷전이고 좋은 술을 만드는 데 전념한 것. 그에 반해 저질 알코올에 물과 감미료를 대충 섞어 “서민들이 선호하는” 술이라고 선전하던 경쟁회사는 급기야 할아버지 친구네 집안의 술에 “공업용 알코올이 들어간다”는 흑색선전을 퍼트리고, “그 술을 마시면 눈이 멀고 불구가 된다”며 비방을 일삼았지만 호소할 방법이 없다. 결국 매출은 떨어지고 공장은 가동을 멈췄으며, 긴 소송 끝에 할아버지 친구네 집안은 몰락하고 만다. 이에 보다 못한 주인공의 할아버지가 ‘저주토끼’를 어여쁘게 만들어 손수 복수에 나서는데…. 러시아를 비롯 슬라브어 권의 명작들을 꾸준히 번역해서 소개하고, 보태어 수준 높은 호러 SF/판타지 창작으로 꾸준히 독자들의 사랑을 받아온 정보라 작가의 대표작. 동서고금을 넘나드는 배경과 인물과 사건들이 넘치는 10편의 작품이 아우르는 주제는 복수와 저주. 저자의 풍부한 상상력은 가차 없는 저주로 복수를 대신한다. 세상의 몹쓸 것들은 도무지 뉘우칠 줄 모르고, 우리의 주인공들인 피해자(혹은 등장토끼 혹은 등장로봇)에게 용서란 없다. 하지만 저자는 말한다. “원래 세상은 쓸쓸한 곳이고 모든 존재는 혼자이며 사필귀정이나 권선징악 혹은 복수는 경우에 따라 반드시 필요할지 모르지만, 그렇게 필요한 일을 완수한 뒤에도 세상은 여전히 쓸쓸하고 인간은 여전히 외롭다는 얘기를 하고 싶었다”고. 그런 사람들에게, 그렇게 고군분투하는 우리 모두에게 위안이 되고 싶었다고. 그렇게 이 책은 악착 같은 저주와 복수에 관한 이야기이자, 위로에 관한 우화들이다. “용서할 수 없는 것들은, 용서하지 말자.”'

In [56]:
rm_ls = ['Foreign', 'Josa', 'Number', 'Suffix', 'Punctuation', 'Determiner', 'VerbPrefix', 'Alpha']
sep_ls = []

for i in tqdm.tqdm(range(df.shape[0])):

    rm_josa = []

    try:
        okt = Okt()
        text = df['story'][i]
        story_sep = okt.pos(text)

        for idx, (a, b) in enumerate(story_sep):
            if b not in rm_ls:
                rm_josa.append((a, b))
    except:
        pass

    sep_ls.append(rm_josa)
    

100%|██████████| 1000/1000 [00:35<00:00, 28.51it/s]


In [71]:
sentence_ls = []
for i in tqdm.tqdm(range(df.shape[0])):
    sentence = ''
    for a, b in sep_ls[i]:
        sentence += a + ' '
    sentence_ls.append(sentence)

100%|██████████| 1000/1000 [00:00<00:00, 11888.65it/s]


In [78]:
df['story_sep'] = sentence_ls # 품사 태깅 포함해서 나눔
df

,itemId,title,story,story_sep
0,281536876,파친코 1,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,어디 속 하지 했던 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 작가 이민 진의 ...
1,281536885,파친코 2,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,어디 속 하지 했던 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 작가 이민 진의 ...
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),불편한데 자꾸 가고 싶은 편의점이 있다! 힘들게 살아낸 오늘을 위로하는 편의점의 밤...,불편한데 자꾸 가고 싶은 편의점 있다 힘들게 살 낸 오늘 위로 하는 편의점 밤 정체...
3,354595636,저주토끼,2022 부커상 최종 후보 지명작! 한국 호러 SF/판타지 대표작가 정보라 대표작 ...,부커상 최종 후보 지 명작 한국 호러 판타지 대표 작가 정보라 대표 작 인터내셔널 ...
4,266107628,아몬드,괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...,괴물 내 또 다른 괴물 만났다 영화 같은 강렬한 사건 매혹 문체 시선 사로잡는 한국...
...,...,...,...,...
995,288668147,하얀 늑대들 3,오랜 기간 사랑 받아온 한국의 대표적인 판타지 소설\r윤현승 작가의 대표작 <하얀 ...,오랜 기간 사랑 받아 온 한국 대표 판타지 소설 윤 현승 작가 대표 작 하얀 늑대 ...
996,289078215,네버무어: 모리건 크로우와 원드러스 평가전 1,"불행의 날에 태어난 저주받은 아이, 운명을 거부하고 신비의 도시 네버무어로 떠나다!...",불행 날 태어난 저주받은 아이 운명 거부 신비 도시 버 무어 떠나다 제시카 타운센드...
997,243131474,춘분 지나고까지,"《아사히 신문》, ‘지난 천 년간의 일본 문학자’ 투표 1위\r무라카미 하루키와 강...",아사히 신문 지난 천 년 일본 문학자 투표 위 무라카미 하루키 강상중 가장 좋아하는...
998,268911218,별의 계승자 2: 가니메데의 친절한 거인,인류의 기원으로 상상하는 인류의 미래\r 2천5백만 년만에 가니메데의 거인이 돌아왔...,인류 기원 상상 하는 인류 미래 백만 년 가니메데 거인 돌아왔다 인류 최초 지적 생...


In [79]:
df.to_csv('novel_story_sep.csv', index=False)